## パート１：数独問題を解けるための準備コード
### Input 1 は　必要なパッケージを導入（１行目から３行目）、そしてデータの保存ルートを変量pathに保存する。
### Input 2 はデータをpandasで　読み取ってから、input 3 のfor 分で内容を表現する

In [1]:
import pandas as pd
from pulp import *
from itertools import product
path=r'C:\Users\84280\Downloads\hard_problem.xlsx'

In [2]:
X=pd.read_excel(path)

In [3]:
for i in range(9):
    print(*X.iloc[i,:])

nan 8.0 nan nan nan nan 9.0 3.0 nan
2.0 nan nan nan nan nan nan 4.0 nan
nan nan 4.0 3.0 8.0 nan nan nan nan
nan nan 3.0 2.0 nan 1.0 nan nan 8.0
nan nan 1.0 nan nan nan nan nan 7.0
nan nan nan 5.0 nan nan nan nan nan
1.0 nan nan nan nan nan nan nan nan
8.0 9.0 nan nan nan nan nan nan 6.0
nan nan nan 4.0 6.0 nan nan 2.0 nan


 ## パート２：数独問題をpulpで設定する
### input 4 は for 文で、LP問題の変数を決めます、X(i,j,k)はそれぞれ９ｘ９数独グラフの行、列と はめる数値です。
### input5　は目標式を定義する。計算のため、０にしました。

In [4]:
prob2=LpProblem('sudoku',sense=LpMinimize)
x={}
for i in range(9):
    for j in range(9):
        for k in range(1,10):
            x[i,j,k]=LpVariable('x('+str(i)+','+str(j)+','+str(k)+')',cat='Binary')
            

In [5]:
prob2+=0

### input 6 は制約条件　どんなマスにも　１～～９の中で、いずれの数値だけ入ります。
$$ \sum_{k=1}^9x_{abk}=1,for a,b,in [0,9] $$

In [6]:
for a in range(9):
    for b in range(9):
        prob2+=lpSum([x[a,b,j] for j in range(1,10)])==1


### input 7 とinput 8はブロック（９マス入る）のためのコード
#### 1. 先ず、プロダクトを作るために、１～９までの数値を均等に三つのリストに保存してから、まとめて、リストtotに入れる。
#### 2. block の空のリストを用意して、for 文を利用して、三つの組それぞれプロダクトを作る。９組が生み出されるはずです。

In [7]:
b_1=[0,1,2]
b_2=[3,4,5]
b_3=[6,7,8]
tot=[b_1,b_2,b_3]

In [8]:
block=[]

for j in tot:
    for k in tot:
        block.append(list(product(j,k)))
block

[[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)],
 [(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)],
 [(0, 6), (0, 7), (0, 8), (1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8)],
 [(3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2)],
 [(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)],
 [(3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (5, 6), (5, 7), (5, 8)],
 [(6, 0), (6, 1), (6, 2), (7, 0), (7, 1), (7, 2), (8, 0), (8, 1), (8, 2)],
 [(6, 3), (6, 4), (6, 5), (7, 3), (7, 4), (7, 5), (8, 3), (8, 4), (8, 5)],
 [(6, 6), (6, 7), (6, 8), (7, 6), (7, 7), (7, 8), (8, 6), (8, 7), (8, 8)]]

### input9 は制約条件、一つブロックの中に含まれるすべての(i,j)について、
$$\sum_{i=0}^8\sum_{j=0}^8x_{ijk}=1,\forall k \in[1,9]\$$

In [9]:
for k in range(1,10):
    for i in block:
        prob2+=lpSum([x[i[j][0],i[j][1],k] for j in range(9)])==1
        


### input 11 は 制約条件、すでにマスに入っている数値をそのまま採用する部分です。
#### 1. 先ずは元のグラフのNaNを全部０で入れ替える。
#### 2. for 文を使って、判定条件 $x_{ij}=0$で無ければ、制約条件の中で $x_{ijk},k=k'$、 $k'$はグラフにすでに決まった値である。

In [11]:
X=X.fillna(0)
print(X.iloc[1,1])
for i in range(9):
    for j in range(9):
        if X.iloc[i,j]!=0:
            prob2+=x[i,j,X.iloc[i,j]]==1



0.0


#### input12 はそれぞれ、すべての行と列に、下記の制限を与える：
行：$$\sum_{j=0}^8x_{ijk}=1,\forall i \in[0,8], \forall k \in[1,9]$$ 
列： $$\sum_{i=0}^8x_{ijk}=1,\forall j \in[0,8],\forall k \in[1,9] $$
#### これで五つの制約条件が満たされる

In [13]:
for k in range(1,10):
    for i in range(9):
        prob2+=lpSum([x[i,j,k] for j in range(9)])==1

for k in range(1,10):
    for j in range(9):
        prob2+=lpSum(x[i,j,k] for i in range(9))==1
prob2

sudoku:
MINIMIZE
0*__dummy + 0
SUBJECT TO
_C1: x(0,0,1) + x(0,0,2) + x(0,0,3) + x(0,0,4) + x(0,0,5) + x(0,0,6)
 + x(0,0,7) + x(0,0,8) + x(0,0,9) = 1

_C2: x(0,1,1) + x(0,1,2) + x(0,1,3) + x(0,1,4) + x(0,1,5) + x(0,1,6)
 + x(0,1,7) + x(0,1,8) + x(0,1,9) = 1

_C3: x(0,2,1) + x(0,2,2) + x(0,2,3) + x(0,2,4) + x(0,2,5) + x(0,2,6)
 + x(0,2,7) + x(0,2,8) + x(0,2,9) = 1

_C4: x(0,3,1) + x(0,3,2) + x(0,3,3) + x(0,3,4) + x(0,3,5) + x(0,3,6)
 + x(0,3,7) + x(0,3,8) + x(0,3,9) = 1

_C5: x(0,4,1) + x(0,4,2) + x(0,4,3) + x(0,4,4) + x(0,4,5) + x(0,4,6)
 + x(0,4,7) + x(0,4,8) + x(0,4,9) = 1

_C6: x(0,5,1) + x(0,5,2) + x(0,5,3) + x(0,5,4) + x(0,5,5) + x(0,5,6)
 + x(0,5,7) + x(0,5,8) + x(0,5,9) = 1

_C7: x(0,6,1) + x(0,6,2) + x(0,6,3) + x(0,6,4) + x(0,6,5) + x(0,6,6)
 + x(0,6,7) + x(0,6,8) + x(0,6,9) = 1

_C8: x(0,7,1) + x(0,7,2) + x(0,7,3) + x(0,7,4) + x(0,7,5) + x(0,7,6)
 + x(0,7,7) + x(0,7,8) + x(0,7,9) = 1

_C9: x(0,8,1) + x(0,8,2) + x(0,8,3) + x(0,8,4) + x(0,8,5) + x(0,8,6)
 + x(0,8,7) + x(0,8,8) + 

### 数独問題を解決するパート
#### input 14は問題を解ける

In [14]:
prob2.solve()


1

#### input 15　はi判断条件 $x_{xijk}=1$を使って、条件満たせば、元のグラフの [i,j]の座標の処にｋを入れる。

In [15]:
for i in range(9):
    for j in range(9):
        for k in range(1,10):
            if value(x[i,j,k])==1:
                print(x[i,j,k])
                X.iloc[i,j]=k

x(0,0,6)
x(0,1,8)
x(0,2,7)
x(0,3,1)
x(0,4,4)
x(0,5,2)
x(0,6,9)
x(0,7,3)
x(0,8,5)
x(1,0,2)
x(1,1,3)
x(1,2,9)
x(1,3,6)
x(1,4,5)
x(1,5,7)
x(1,6,8)
x(1,7,4)
x(1,8,1)
x(2,0,5)
x(2,1,1)
x(2,2,4)
x(2,3,3)
x(2,4,8)
x(2,5,9)
x(2,6,6)
x(2,7,7)
x(2,8,2)
x(3,0,4)
x(3,1,6)
x(3,2,3)
x(3,3,2)
x(3,4,7)
x(3,5,1)
x(3,6,5)
x(3,7,9)
x(3,8,8)
x(4,0,9)
x(4,1,5)
x(4,2,1)
x(4,3,8)
x(4,4,3)
x(4,5,4)
x(4,6,2)
x(4,7,6)
x(4,8,7)
x(5,0,7)
x(5,1,2)
x(5,2,8)
x(5,3,5)
x(5,4,9)
x(5,5,6)
x(5,6,3)
x(5,7,1)
x(5,8,4)
x(6,0,1)
x(6,1,4)
x(6,2,6)
x(6,3,9)
x(6,4,2)
x(6,5,5)
x(6,6,7)
x(6,7,8)
x(6,8,3)
x(7,0,8)
x(7,1,9)
x(7,2,2)
x(7,3,7)
x(7,4,1)
x(7,5,3)
x(7,6,4)
x(7,7,5)
x(7,8,6)
x(8,0,3)
x(8,1,7)
x(8,2,5)
x(8,3,4)
x(8,4,6)
x(8,5,8)
x(8,6,1)
x(8,7,2)
x(8,8,9)


#### input 16は結果を出力する

In [16]:

X

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,6.0,8.0,7.0,1.0,4.0,2.0,9.0,3.0,5.0
1,2.0,3.0,9.0,6.0,5.0,7.0,8.0,4.0,1.0
2,5.0,1.0,4.0,3.0,8.0,9.0,6.0,7.0,2.0
3,4.0,6.0,3.0,2.0,7.0,1.0,5.0,9.0,8.0
4,9.0,5.0,1.0,8.0,3.0,4.0,2.0,6.0,7.0
5,7.0,2.0,8.0,5.0,9.0,6.0,3.0,1.0,4.0
6,1.0,4.0,6.0,9.0,2.0,5.0,7.0,8.0,3.0
7,8.0,9.0,2.0,7.0,1.0,3.0,4.0,5.0,6.0
8,3.0,7.0,5.0,4.0,6.0,8.0,1.0,2.0,9.0


### 最後のパートは検証するパート、問題なさそうです

In [29]:
##check
for i in range(9):
    print('coloumn'+str(i),X.iloc[:,i].is_unique)
    print('row'+str(i),X.iloc[:,i].is_unique)

coloumn0 True
row0 True
coloumn1 True
row1 True
coloumn2 True
row2 True
coloumn3 True
row3 True
coloumn4 True
row4 True
coloumn5 True
row5 True
coloumn6 True
row6 True
coloumn7 True
row7 True
coloumn8 True
row8 True


In [35]:
for i in block:
    a=list(X.iloc[i[j][0],i[j][1]] for j in range(9))
    print(pd.Series(a).is_unique)
        

True
True
True
True
True
True
True
True
True


### 応用できるところ： 中小学校で、席を決めるときに、目が悪い人を前にする条件と、背が高い人が後ろにする条件と、それぞれの希望度に踏まえて、希望度最大を目標関数として、座る席を決めます